In [1]:
from main import *
%load_ext autoreload
%autoreload 2

In [2]:
conf = SparkConf().setAppName('playground').setMaster('local[*]')
sc = SparkContext(conf=conf)

In [4]:
# load static resources
rules = load_rules()
doclogfreq = load_unigram_cts()
(entid2name, entname2id, entname_set) = load_ent2name()
(doclogfreq, filter_words) = load_unigram_cts()
(primes_map, ent_spell_scores, possible_spelling_ratios) = make_mispelling_resources(entname_set)

In [6]:
with open('train.txt','r') as f :
    lines = f.read().split('\n')
    lines = [line.split('\t') for line in lines]

relationship_bags = defaultdict(dict)
    
for line in lines :
    id1 = line[0].replace('www.freebase.com/m','')
    rel_type = line[1].replace('www.freebase.com','')
    id2 = line[2].replace('www.freebase.com/m','')
    text = filter(lambda c: c in [' ','-','\''] or (c>='a' and c<='z') or (c>='0' and c<='9'), line[3].lower())
    text = text.replace('\'',' \'')
    if id1 not in entid2name :
        continue
    ent = entid2name[id1]
    new_text = (' '+text+' ').replace(' '+ent+' ',' ')
    words = set(new_text.split()) # TODO using set might be sketchy but dont really want to double count duplicates
    word_weights = dict( zip(words,[1.0/len(words) for i in range(len(words))]) )
    relationship_bags[rel_type] = outer_join_dicts(op.add, relationship_bags[rel_type], word_weights)

In [7]:
# normalize each bag to 1
relationship_bags_norm = {}
relationship_counts = {}
for rel_type in relationship_bags :
    sm = reduce(op.add,relationship_bags[rel_type].values())
    relationship_bags_norm[rel_type] = map_values(lambda x: x/sm, relationship_bags[rel_type])
    relationship_counts[rel_type] = sm

In [8]:
def tfidf_dotproduct(bag1,bag2) :
    score = 0.0
    for key in set.intersection(set(bag1.keys()),set(bag2.keys())) :
        score += bag1[key]*bag2[key]
    return score

In [ ]:
with open('test.txt','r') as f :
    lines = f.read().split('\n')
    lines = [line.split('\t') for line in lines]

for line in lines[1000:2000] :
    
    # process line
    id1 = line[0].replace('www.freebase.com/m','')
    rel_true = line[1].replace('www.freebase.com','')
    id2 = line[2].replace('www.freebase.com/m','')
    text = filter(lambda c: c in [' ','-','\''] or (c>='a' and c<='z') or (c>='0' and c<='9'), line[3].lower())
    text = re.sub(r"'(?=[a-z])", r" '", text)
    
    # for weird reason, ent id is not in dataset so auto wrong answer
    if id1 not in entid2name :
        print text
        print ':( nonono'
        print ''
        continue
    print text,'|',entid2name[id1],'|',rel_true
    
    # generate list of all possible entity spellings
    words = text.split()
    unigrams = filter(lambda tup: tup[0] not in filter_words, list(ngrams(words,1)))
    grams_list = unigrams
    for i in range(len(words)) :
        grams_list += list(ngrams(words,i))
    grams = set(grams_list)
    
    # exact match entities
    possible_ents = list(set.intersection(grams,entname_set))
    P_spelling = [1.0] * len(possible_ents)
    
    # get possible mispellings
    mispelled_ents = []
    for ent in grams-set(possible_ents) :
        if len(' '.join(ent))<=4 or all([doclogfreq[w]>12 for w in ent]) :
            continue
        poss_ents = find_mispellings(ent, primes_map, ent_spell_scores, possible_spelling_ratios)
        mispelled_ents += poss_ents
    possible_ents += mispelled_ents
    P_spelling += [0.5] * len(mispelled_ents)
    
    predictions = []
    for ent_i in range(len(possible_ents)) :
        ent_words = possible_ents[ent_i]
        P_ent_len = math.pow(0.5,6-len(ent_words))
        ent_freq = reduce(op.add,[math.log(360000000)-doclogfreq[w] for w in ent_words])
        #print ent_words, P_ent_len, ent_freq
        #new_text = (' '+text+' ').replace(' '+' '.join(ent)+' ',' CHINCHILLA ')
        new_text = (' '+text+' ').replace(' '+' '.join(ent_words)+' ',' ')
        words = set(new_text.split()) # TODO using set might be sketchy but dont really want to double count duplicates
        word_weights = dict( zip(words,[1.0/len(words) for i in range(len(words))]) )
        P_rel = dict([(k,tfidf_dotproduct(word_weights,relationship_bags_norm[k])) for k in relationship_bags_norm])
        #P_rel.sort(lambda p1,p2: int((p2[1]-p1[1])*10000))
        #print P_rel[:10]

        # get all possible rules for entity
        ids = entname2id[' '.join(ent_words)]
        #if len(ids)>100 :
        #    continue
        rules_list = []
        for id in ids :
            rules_list += rules[id]
        #print 'yo',rules_for_id
        for rel in rules_list :
            if rel not in P_rel :
                continue
            #print rel, P_rel[rel]
            predictions.append( (' '.join(ent_words), rel, P_spelling[ent_i]*P_rel[rel]*ent_freq) )
        
    #if id1 not in entid2name_important :
    #    continue
    #ent = entid2name_important[id1]
    #print ent
    #new_text = (' '+text+' ').replace(' '+ent+' ',' ')
    predictions.sort(lambda p1,p2: int(p2[2]*1000-p1[2]*1000))
    #print ''
    for i in range(min(5,len(predictions))) :
        print predictions[i]
    if predictions[0][0]==entid2name[id1] and predictions[0][1]==rel_true :
        print 'yesyesyes'
    else :
        print 'nonono'
    print ''

who is the lyricist behind the composition supercalifragilisticexpialidocious | supercalifragilisticexpialidocious | /music/composition/lyricist
('supercalifragilisticexpialidocious', '/music/composition/lyricist', 1.2343440036943165)
('supercalifragilisticexpialidocious', '/music/composition/composer', 1.1385400430124137)
('supercalifragilisticexpialidocious', '/music/recording/artist', 0.6915690804042466)
('supercalifragilisticexpialidocious', '/music/recording/artist', 0.6915690804042466)
('supercalifragilisticexpialidocious', '/music/recording/artist', 0.6915690804042466)
yesyesyes

what romance film was released in 1950 | romance film | /film/film_genre/films_in_this_genre
('romance film', '/base/skosbase/vocabulary_equivalent_topic/broader_concept', 1.00777023857554)
('romance film', '/user/micahsaul/advertising/advertised_thing/ad_campaigns', 0.9797766208373305)
('romance film', '/type/object/type', 0.884776057782899)
('romance film', '/common/topic/notable_types', 0.85271444079

In [ ]:
14 unsolvable/weird/indistinguishable
1 ent
16 exact lm (credit for full match or single word of two in any part of rel, sometimes plural vs singular)
9 lm (almost always synonym)

In [ ]:
59 correct
17 ?ent
19 nope5
5 nopeno (almost all really similar rules chosen)

In [ ]:
x13 common ent
carlos gomez, pretty in pink, somevelvetsidewalk
-2 not exact match (ED: 1,2)

In [35]:
for line in lines[:100] :
    id1 = line[0].replace('www.freebase.com/m','')
    print entid2name_important[id1] if id1 in entid2name_important else ':('

harderfaster
fearless
alex golfis
phil hay
roger marquis
yves klein
dublin
action
las vegas
akbar tandjung
jean grae
carlos gmez
engelbert zaschka
carolyn s shoemaker
pretty in pink
woodstock villa
shawn holly cookson
:(
pee wee reese
one life to live
stephen mao
carpenter
myocardial infarction
david ruffin
equator
los angeles
history
padraic mcguinness
viper
environmental disaster
magical mystery tour
michael mcdowell
the complete on the corner sessions
michael dotson
sam edwards
second battle of fort fisher
pembroke welsh corgi
music for cello orchestra
voodoo lounge
some velvet sidewalk
tokyo
italianate architecture
loud
neo contra
western
hindi language
aguascalientes
herb alpert
angie estes
indiana
jeff hall
jos francisco torres
philip leacock
chic murray
the soul sessions
frei otto
cow belles
around the world
a knot in the grain and other stories
dancin' homer
british columbia
primero river
johannes messenius
the house that never was
roman italy
taffin
warsaw
philharmonia orchest

In [99]:
rules['/02ppyz8']

['/people/person/profession',
 '/people/person/nationality',
 '/people/person/gender',
 '/people/person/place_of_birth']

In [123]:
find_mispellings(('somevelvetsidewalk',))

[('some', 'velvet', 'sidewalk')]

In [104]:
len(possible_composites)

780

In [97]:
ent_spell_scores[20911329313528897743071110L]

KeyError: 20911329313528897743071110L

In [98]:
num_list = [primes_map[c] for c in ' '.join(('skyclan\'s','destiny',))]
reduce(op.mul,num_list)

212803527720029371150076590L

In [99]:
num_list = [primes_map[c] for c in ' '.join(('skyclans','destiny',))]
reduce(op.mul,num_list)

1230078194913464573121830L

In [115]:
1230078194913464573121830L/173

7110278583314824122091L

In [129]:
doclogfreq['until'],doclogfreq['areas'],doclogfreq['gomez'],doclogfreq['random']

(13.162954305230732, 12.283945308050269, 8.902591637374087, 10.364860292582588)